In [1]:
import pandas as pd
import thinkplot
import numpy as np
import re
import thinkstats2

Read in cleaned time series data and find correlations between percent female and pay gap and between percent female and total weekly pay for each occupation.  

In [16]:
df = pd.read_excel('Data/weeklyincome_occupation_gender_combined_clean_all.xlsx')
df = df.set_index('Occupation')
df.head()

,Percent Female,Pay Gap,Total Weekly Pay,Female Weekly Pay,Male Weekly Pay
Occupation,,,,,
Total fulltime wage and salary workers,"[44.497996833005956, 44.39843915530758, 44.252...","[184.0, 175.16842105263163, 173.70345217883425...","[886.0, 880.9639889196677, 870.6100735710244, ...","[789.0, 788.7700831024931, 783.7583474816072, ...","[973.0, 963.9385041551247, 957.4617996604414, ..."
Management professional and related occupations,"[51.47311916079331, 51.672421462918635, 51.543...","[390.0, 399.50692520775624, 411.23769100169784...","[1246.0, 1253.8371191135736, 1243.130730050933...","[1078.0, 1077.64432132964, 1074.6593095642331,...","[1468.0, 1477.1512465373962, 1485.897000565931..."
Management business and financial operations occupations,"[46.29210089110406, 46.35314721335119, 46.0547...","[369.0, 401.5556786703603, 410.19128466327106,...","[1355.0, 1359.3479224376733, 1343.585738539898...","[1168.0, 1161.643213296399, 1150.0005659309563...","[1537.0, 1563.1988919667592, 1560.191850594227..."
Management occupations,"[42.472360317578094, 42.524109651454175, 41.42...","[349.0, 409.75069252077583, 409.1448783248445,...","[1429.0, 1425.9324099722994, 1433.576683644595...","[1236.0, 1201.5939058171746, 1201.274476513865...","[1585.0, 1611.3445983379504, 1610.419354838709..."
Chief executives,"[27.959927140255008, 27.55281690140845, 27.652...","[752.0, 507.06648199446, 568.1986417657045, 43...","[2291.0, 2351.9689750692523, 2409.873797396717...","[1736.0, 1966.8033240997233, 1963.058290888511...","[2488.0, 2473.8698060941833, 2531.256932654216..."


Find correlations and changes in percent female, total weekly pay, and pay gap and add to df. Also find percent increases and decreases for percent female change, total weekly pay change, and pay gap change. If the change is negative, that means the value from 2018 is lower than the value from 2003. We can determine the percent **decrease** by dividing the change by the value from 2003. If the change is positive, that means that the value from 2018 is higher than the value from 2003. We can determine the percent **increase** by dividing the change by the value from 2003. 

In [17]:
df['Correlation1'] = pd.Series() # with total weekly pay
df['Correlation2'] = pd.Series() # with pay gap
df['Percent Female Change'] = pd.Series()
df['Total Weekly Pay Change'] = pd.Series()
df['Pay Gap Change'] = pd.Series()
df['Percent Female Change (%)'] = pd.Series()
df['Total Weekly Pay Change (%)'] = pd.Series()
df['Pay Gap Change (%)'] = pd.Series()
for occ in df.index:
    temp_percent_female = np.asarray(eval(df.loc[occ]['Percent Female']))
    temp_total_pay = np.asarray(eval(df.loc[occ]['Total Weekly Pay']))
    temp_pay_gap = np.asarray(eval(df.loc[occ]['Pay Gap']))
    
    corr1 = thinkstats2.Cov(temp_percent_female, temp_total_pay)
    corr2 = thinkstats2.Cov(temp_percent_female, temp_pay_gap)
    temp_percent_female_change = temp_percent_female[0]-temp_percent_female[-1]
    temp_total_pay_change = temp_total_pay[0]-temp_total_pay[-1]
    temp_pay_gap_change = temp_pay_gap[0]-temp_pay_gap[-1]
    temp_percent_female_change_percent = (temp_percent_female_change/temp_percent_female[-1])*100
    temp_total_pay_change_percent = (temp_total_pay_change/temp_total_pay[-1])*100
    temp_pay_gap_change_percent = (temp_pay_gap_change/temp_pay_gap[-1])*100
    
    df.at[occ, 'Correlation1'] = corr1
    df.at[occ, 'Correlation2'] = corr2
    df.at[occ, 'Percent Female Change'] = temp_percent_female_change
    df.at[occ, 'Total Weekly Pay Change'] = temp_total_pay_change
    df.at[occ, 'Pay Gap Change'] = temp_pay_gap_change
    df.at[occ, 'Percent Female Change (%)'] = temp_percent_female_change_percent
    df.at[occ, 'Total Weekly Pay Change (%)'] = temp_total_pay_change_percent
    df.at[occ, 'Pay Gap Change (%)'] = temp_pay_gap_change_percent

C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in double_scalars


In [18]:
df = df[list(df.columns.values)[::-1]]
df.head()

,Pay Gap Change (%),Total Weekly Pay Change (%),Percent Female Change (%),Pay Gap Change,Total Weekly Pay Change,Percent Female Change,Correlation2,Correlation1,Male Weekly Pay,Female Weekly Pay,Total Weekly Pay,Pay Gap,Percent Female
Occupation,,,,,,,,,,,,,
Total fulltime wage and salary workers,-5.984335,4.414418,1.262321,-11.712065,37.458179,0.554706,-0.270090,2.964479,"[973.0, 963.9385041551247, 957.4617996604414, ...","[789.0, 788.7700831024931, 783.7583474816072, ...","[886.0, 880.9639889196677, 870.6100735710244, ...","[184.0, 175.16842105263163, 173.70345217883425...","[44.497996833005956, 44.39843915530758, 44.252..."
Management professional and related occupations,-5.329091,2.639106,2.247015,-21.953368,32.037750,1.131191,-2.101778,3.082734,"[1468.0, 1477.1512465373962, 1485.897000565931...","[1078.0, 1077.64432132964, 1074.6593095642331,...","[1246.0, 1253.8371191135736, 1243.130730050933...","[390.0, 399.50692520775624, 411.23769100169784...","[51.47311916079331, 51.672421462918635, 51.543..."
Management business and financial operations occupations,-21.623411,3.023036,4.081821,-101.803849,39.760178,1.815457,-12.860920,3.935588,"[1537.0, 1563.1988919667592, 1560.191850594227...","[1168.0, 1161.643213296399, 1150.0005659309563...","[1355.0, 1359.3479224376733, 1343.585738539898...","[369.0, 401.5556786703603, 410.19128466327106,...","[46.29210089110406, 46.35314721335119, 46.0547..."
Management occupations,-21.051962,2.064575,8.131871,-93.062916,28.905996,3.194061,-24.631875,4.211044,"[1585.0, 1611.3445983379504, 1610.419354838709...","[1236.0, 1201.5939058171746, 1201.274476513865...","[1429.0, 1425.9324099722994, 1433.576683644595...","[349.0, 409.75069252077583, 409.1448783248445,...","[42.472360317578094, 42.524109651454175, 41.42..."
Chief executives,11.452383,7.442492,19.173460,77.272391,158.696521,4.498389,-41.754075,36.454728,"[2488.0, 2473.8698060941833, 2531.256932654216...","[1736.0, 1966.8033240997233, 1963.058290888511...","[2291.0, 2351.9689750692523, 2409.873797396717...","[752.0, 507.06648199446, 568.1986417657045, 43...","[27.959927140255008, 27.55281690140845, 27.652..."


In [19]:
#df.to_excel('Data/weeklyincome_occupation_gender_combined_clean_all_calculations.xlsx')

Are there any occupations in which the percent female change is positive?

In [20]:
df_percent_female_pos = df[df['Percent Female Change'] > 0]
print(df_percent_female_pos.shape)
df_percent_female_pos.head()

(59, 13)


,Pay Gap Change (%),Total Weekly Pay Change (%),Percent Female Change (%),Pay Gap Change,Total Weekly Pay Change,Percent Female Change,Correlation2,Correlation1,Male Weekly Pay,Female Weekly Pay,Total Weekly Pay,Pay Gap,Percent Female
Occupation,,,,,,,,,,,,,
Total fulltime wage and salary workers,-5.984335,4.414418,1.262321,-11.712065,37.458179,0.554706,-0.270090,2.964479,"[973.0, 963.9385041551247, 957.4617996604414, ...","[789.0, 788.7700831024931, 783.7583474816072, ...","[886.0, 880.9639889196677, 870.6100735710244, ...","[184.0, 175.16842105263163, 173.70345217883425...","[44.497996833005956, 44.39843915530758, 44.252..."
Management professional and related occupations,-5.329091,2.639106,2.247015,-21.953368,32.037750,1.131191,-2.101778,3.082734,"[1468.0, 1477.1512465373962, 1485.897000565931...","[1078.0, 1077.64432132964, 1074.6593095642331,...","[1246.0, 1253.8371191135736, 1243.130730050933...","[390.0, 399.50692520775624, 411.23769100169784...","[51.47311916079331, 51.672421462918635, 51.543..."
Management business and financial operations occupations,-21.623411,3.023036,4.081821,-101.803849,39.760178,1.815457,-12.860920,3.935588,"[1537.0, 1563.1988919667592, 1560.191850594227...","[1168.0, 1161.643213296399, 1150.0005659309563...","[1355.0, 1359.3479224376733, 1343.585738539898...","[369.0, 401.5556786703603, 410.19128466327106,...","[46.29210089110406, 46.35314721335119, 46.0547..."
Management occupations,-21.051962,2.064575,8.131871,-93.062916,28.905996,3.194061,-24.631875,4.211044,"[1585.0, 1611.3445983379504, 1610.419354838709...","[1236.0, 1201.5939058171746, 1201.274476513865...","[1429.0, 1425.9324099722994, 1433.576683644595...","[349.0, 409.75069252077583, 409.1448783248445,...","[42.472360317578094, 42.524109651454175, 41.42..."
Chief executives,11.452383,7.442492,19.173460,77.272391,158.696521,4.498389,-41.754075,36.454728,"[2488.0, 2473.8698060941833, 2531.256932654216...","[1736.0, 1966.8033240997233, 1963.058290888511...","[2291.0, 2351.9689750692523, 2409.873797396717...","[752.0, 507.06648199446, 568.1986417657045, 43...","[27.959927140255008, 27.55281690140845, 27.652..."


There seems to be quite a lot. Let's order them in descending order.

In [31]:
df_percent_female_pos_sorted = df_percent_female_pos.sort_values(by=['Percent Female Change (%)'], ascending=False)
df_percent_female_pos_sorted.iloc[:10]

,Pay Gap Change (%),Total Weekly Pay Change (%),Percent Female Change (%),Pay Gap Change,Total Weekly Pay Change,Percent Female Change,Correlation2,Correlation1,Male Weekly Pay,Female Weekly Pay,Total Weekly Pay,Pay Gap,Percent Female
Occupation,,,,,,,,,,,,,
Driversales workers and truck drivers,49.454250,-1.002738,43.567904,89.342709,-8.275352,1.685315,6.670935,2.602990,"[829.0, 826.6720221606649, 823.5217883418223, ...","[559.0, 603.3578947368421, 659.2359932088285, ...","[817.0, 816.4282548476456, 816.1969439728354, ...","[270.0, 223.3141274238228, 164.28579513299383,...","[5.553565030455034, 4.915134370579914, 4.27198..."
Physicians and surgeons,-11.215679,4.061289,35.336582,-105.607698,78.094745,11.113388,-332.909998,-132.076927,"[2513.0, 2332.5058171745154, 2451.730050933786...","[1677.0, 1801.878670360111, 1544.4957555178269...","[2001.0, 1964.7545706371193, 2004.91454442558,...","[836.0, 530.6271468144043, 907.2342954159592, ...","[42.563482466747274, 43.190184049079754, 38.21..."
Pharmacists,64.723935,2.451434,32.257733,149.703923,49.554404,15.301745,-57.549987,-118.863048,"[2271.0, 2282.3113573407204, 2193.267685342388...","[1890.0, 1878.7069252077565, 1924.341256366723...","[2071.0, 1969.876454293629, 2013.2857951329938...","[381.0, 403.6044321329639, 268.926428975665, 3...","[62.737642585551335, 57.874015748031496, 59.90..."
Laborers and freight stock and material movers hand,-3.795745,-5.045015,31.627960,-3.116950,-32.037750,4.790595,-7.921527,-1.324972,"[620.0, 609.5041551246538, 606.9156762874929, ...","[541.0, 512.1883656509696, 532.6208262591964, ...","[603.0, 592.0897506925209, 592.265987549519, 5...","[79.0, 97.31578947368428, 74.29485002829654, 9...","[19.93730407523511, 17.517241379310345, 15.596..."
Lawyers,56.064417,-8.807359,27.998369,158.065137,-188.040711,8.821404,1.742563,-106.336064,"[2202.0, 2156.3130193905818, 2182.803621958121...","[1762.0, 1795.7324099722994, 1694.131861912846...","[1947.0, 1947.3401662049864, 1985.032823995472...","[440.0, 360.5806094182824, 488.6717600452746, ...","[40.328253223915596, 43.149807938540334, 40.13..."
Postal service mail carriers,652.155378,-12.451824,27.660819,151.733531,-132.414508,8.877628,55.922125,-129.108239,"[983.0, 988.5235457063714, 1067.3344651952461,...","[808.0, 876.86648199446, 974.2043010752689, 90...","[931.0, 967.0116343490306, 1026.5246179966043,...","[175.0, 111.6570637119114, 93.13016411997728, ...","[40.97222222222222, 40.6015037593985, 39.14473..."
Transportation and material moving occupations,-0.800196,-3.186858,27.390746,-1.500370,-22.680237,3.526543,-0.366731,-1.788052,"[724.0, 727.3074792243768, 716.7883418222976, ...","[538.0, 526.5296398891967, 565.0594227504245, ...","[689.0, 697.6005540166205, 692.7209960384833, ...","[186.0, 200.77783933518015, 151.72891907187318...","[16.40148501723681, 14.856274184632392, 14.056..."
Construction and extraction occupations,-83.299081,-1.439482,26.147258,-119.704663,-11.800888,0.617238,-1.560706,-0.086329,"[809.0, 815.4038781163435, 822.4753820033956, ...","[785.0, 821.5501385041553, 668.653650254669, 7...","[808.0, 815.4038781163435, 820.3825693265421, ...","[24.0, -6.146260387811708, 153.82173174872662,...","[2.977860929217337, 2.6028957214901576, 2.5255..."
Police and sheriffs patrol officers,490.152681,4.435396,25.768434,261.623982,46.377498,2.983258,29.741338,1.479430,"[1131.0, 1048.9617728531857, 1054.777589134125...","[816.0, 1009.0110803324101, 981.5291454442558,...","[1092.0, 1039.7423822714682, 1050.591963780418...","[315.0, 39.95069252077565, 73.24844368986987, ...","[14.560439560439562, 13.662790697674417, 14.10..."


Are there any occupations in which percent female change is positive, but total weekly pay change is negative?

In [22]:
df_percent_female_pos_total_pay_neg = df_percent_female_pos[df_percent_female_pos['Total Weekly Pay Change'] < 0]
print(df_percent_female_pos_total_pay_neg.shape)
df_percent_female_pos_total_pay_neg.head()

(24, 13)


,Pay Gap Change (%),Total Weekly Pay Change (%),Percent Female Change (%),Pay Gap Change,Total Weekly Pay Change,Percent Female Change,Correlation2,Correlation1,Male Weekly Pay,Female Weekly Pay,Total Weekly Pay,Pay Gap,Percent Female
Occupation,,,,,,,,,,,,,
General and operations managers,3.152738,-13.941015,15.327461,8.802369,-216.747594,4.005515,-18.822717,-75.799645,"[1427.0, 1524.2725761772856, 1421.019807583474...","[1139.0, 1161.643213296399, 1085.1233729485002...","[1338.0, 1360.3722991689751, 1324.750424448217...","[288.0, 362.6293628808867, 335.89643463497464,...","[30.138445154419596, 34.891304347826086, 28.53..."
Purchasing managers,-67.418460,-10.183571,15.259875,-417.982976,-154.426351,5.904118,-77.976662,-104.907715,"[1431.0, 1454.6149584487537, 1464.968873797396...","[1229.0, 1165.7407202216068, 1223.249009620826...","[1362.0, 1301.9828254847646, 1333.121675155631...","[202.0, 288.87423822714686, 241.71986417657058...","[44.5945945945946, 53.608247422680414, 49.1978..."
Food service managers,28.165867,-8.779604,14.757131,47.028868,-77.863064,6.328651,-11.476641,-62.827088,"[922.0, 915.7927977839337, 892.5846066779853, ...","[708.0, 690.429916897507, 661.328805885682, 72...","[809.0, 771.3556786703602, 773.2942840973401, ...","[214.0, 225.36288088642664, 231.25580079230338...","[49.21402660217654, 48.28431372549019, 46.0526..."
Personal financial advisors,-27.264095,-1.620670,16.817633,-164.928201,-24.265729,5.077021,252.383818,-54.114987,"[1647.0, 1702.5141274238229, 1793.540464063384...","[1207.0, 1002.8648199445985, 997.2252405206565...","[1473.0, 1459.7368421052633, 1469.154499151103...","[440.0, 699.6493074792244, 796.3152235427277, ...","[35.26570048309179, 32.914572864321606, 35.588..."
Social workers,-10.858843,-4.232174,5.265425,-7.430792,-40.082161,4.058765,-0.664509,-4.993271,"[958.0, 957.7922437673132, 1087.2161856253538,...","[897.0, 921.9390581717453, 925.0232031692134, ...","[907.0, 928.0853185595569, 948.044142614601, 9...","[61.0, 35.85318559556788, 162.19298245614038, ...","[81.14209827357237, 82.77310924369748, 81.4327..."


There are a few. Let's order them in ascending order by `Total Weekly Pay Change (%)`.

In [24]:
df_percent_female_pos_total_pay_neg_sorted = df_percent_female_pos_total_pay_neg.sort_values(by=['Total Weekly Pay Change (%)'], ascending=True)
df_percent_female_pos_total_pay_neg_sorted.iloc[:10]

,Pay Gap Change (%),Total Weekly Pay Change (%),Percent Female Change (%),Pay Gap Change,Total Weekly Pay Change,Percent Female Change,Correlation2,Correlation1,Male Weekly Pay,Female Weekly Pay,Total Weekly Pay,Pay Gap,Percent Female
Occupation,,,,,,,,,,,,,
Insurance sales agents,-53.056723,-15.526053,16.301977,-270.125093,-157.881569,7.850689,-120.508469,-49.972788,"[1048.0, 1083.7905817174517, 1220.109790605546...","[809.0, 814.3795013850416, 707.3706847764573, ...","[859.0, 918.8659279778394, 877.9349179400114, ...","[239.0, 269.41108033241005, 512.7391058290888,...","[56.00858369098712, 55.00000000000001, 48.4633..."
General and operations managers,3.152738,-13.941015,15.327461,8.802369,-216.747594,4.005515,-18.822717,-75.799645,"[1427.0, 1524.2725761772856, 1421.019807583474...","[1139.0, 1161.643213296399, 1085.1233729485002...","[1338.0, 1360.3722991689751, 1324.750424448217...","[288.0, 362.6293628808867, 335.89643463497464,...","[30.138445154419596, 34.891304347826086, 28.53..."
Bailiffs correctional officers and jailers,-19.626825,-13.807059,4.635319,-40.292376,-118.859363,1.180577,-6.401160,-9.166187,"[811.0, 845.1108033240998, 829.8002263723826, ...","[646.0, 685.3080332409974, 703.1850594227504, ...","[742.0, 789.7944598337951, 783.7583474816072, ...","[165.0, 159.80277008310247, 126.61516694963211...","[26.649746192893403, 27.17948717948718, 24.931..."
Clinical laboratory technologists and technicians,-193.362779,-13.117970,2.822250,-190.540340,-134.830496,2.034927,-43.028411,26.349535,"[819.0, 1086.8637119113575, 1120.7011884550084...","[911.0, 865.5983379501386, 872.7028862478777, ...","[893.0, 911.6952908587259, 928.1624221844935, ...","[-92.0, 221.26537396121898, 247.99830220713068...","[74.13793103448276, 68.35016835016835, 67.1186..."
Postal service mail carriers,652.155378,-12.451824,27.660819,151.733531,-132.414508,8.877628,55.922125,-129.108239,"[983.0, 988.5235457063714, 1067.3344651952461,...","[808.0, 876.86648199446, 974.2043010752689, 90...","[931.0, 967.0116343490306, 1026.5246179966043,...","[175.0, 111.6570637119114, 93.13016411997728, ...","[40.97222222222222, 40.6015037593985, 39.14473..."
Purchasing managers,-67.418460,-10.183571,15.259875,-417.982976,-154.426351,5.904118,-77.976662,-104.907715,"[1431.0, 1454.6149584487537, 1464.968873797396...","[1229.0, 1165.7407202216068, 1223.249009620826...","[1362.0, 1301.9828254847646, 1333.121675155631...","[202.0, 288.87423822714686, 241.71986417657058...","[44.5945945945946, 53.608247422680414, 49.1978..."
Lawyers,56.064417,-8.807359,27.998369,158.065137,-188.040711,8.821404,1.742563,-106.336064,"[2202.0, 2156.3130193905818, 2182.803621958121...","[1762.0, 1795.7324099722994, 1694.131861912846...","[1947.0, 1947.3401662049864, 1985.032823995472...","[440.0, 360.5806094182824, 488.6717600452746, ...","[40.328253223915596, 43.149807938540334, 40.13..."
Food service managers,28.165867,-8.779604,14.757131,47.028868,-77.863064,6.328651,-11.476641,-62.827088,"[922.0, 915.7927977839337, 892.5846066779853, ...","[708.0, 690.429916897507, 661.328805885682, 72...","[809.0, 771.3556786703602, 773.2942840973401, ...","[214.0, 225.36288088642664, 231.25580079230338...","[49.21402660217654, 48.28431372549019, 46.0526..."
Laborers and freight stock and material movers hand,-3.795745,-5.045015,31.627960,-3.116950,-32.037750,4.790595,-7.921527,-1.324972,"[620.0, 609.5041551246538, 606.9156762874929, ...","[541.0, 512.1883656509696, 532.6208262591964, ...","[603.0, 592.0897506925209, 592.265987549519, 5...","[79.0, 97.31578947368428, 74.29485002829654, 9...","[19.93730407523511, 17.517241379310345, 15.596..."


TODO: Analyze
- Postal service mail carriers

Are there any occupations in which percent female change is positive, and pay gap change is positive? (Difference in pay between men and women greater in 2018 than in 2003)

In [25]:
df_percent_female_pos_pay_gap_pos = df_percent_female_pos[df_percent_female_pos['Pay Gap Change'] > 0]
print(df_percent_female_pos_pay_gap_pos.shape)
df_percent_female_pos_pay_gap_pos.head()

(26, 13)


,Pay Gap Change (%),Total Weekly Pay Change (%),Percent Female Change (%),Pay Gap Change,Total Weekly Pay Change,Percent Female Change,Correlation2,Correlation1,Male Weekly Pay,Female Weekly Pay,Total Weekly Pay,Pay Gap,Percent Female
Occupation,,,,,,,,,,,,,
Chief executives,11.452383,7.442492,19.173460,77.272391,158.696521,4.498389,-41.754075,36.454728,"[2488.0, 2473.8698060941833, 2531.256932654216...","[1736.0, 1966.8033240997233, 1963.058290888511...","[2291.0, 2351.9689750692523, 2409.873797396717...","[752.0, 507.06648199446, 568.1986417657045, 43...","[27.959927140255008, 27.55281690140845, 27.652..."
General and operations managers,3.152738,-13.941015,15.327461,8.802369,-216.747594,4.005515,-18.822717,-75.799645,"[1427.0, 1524.2725761772856, 1421.019807583474...","[1139.0, 1161.643213296399, 1085.1233729485002...","[1338.0, 1360.3722991689751, 1324.750424448217...","[288.0, 362.6293628808867, 335.89643463497464,...","[30.138445154419596, 34.891304347826086, 28.53..."
Food service managers,28.165867,-8.779604,14.757131,47.028868,-77.863064,6.328651,-11.476641,-62.827088,"[922.0, 915.7927977839337, 892.5846066779853, ...","[708.0, 690.429916897507, 661.328805885682, 72...","[809.0, 771.3556786703602, 773.2942840973401, ...","[214.0, 225.36288088642664, 231.25580079230338...","[49.21402660217654, 48.28431372549019, 46.0526..."
Medical and health services managers,49.130646,4.338401,7.607290,131.119911,58.544782,5.211725,-15.263389,-6.129977,"[1729.0, 1526.3213296398894, 1684.714204867006...","[1331.0, 1205.6914127423825, 1312.193548387096...","[1408.0, 1351.1529085872578, 1467.06168647425,...","[398.0, 320.629916897507, 372.52065647990935, ...","[73.72134038800705, 70.98360655737706, 74.8592..."
Social and community service managers,161.353330,5.798099,9.445825,172.247964,62.530718,5.967711,128.301908,22.466199,"[1336.0, 1312.2265927977842, 1261.966044142614...","[1057.0, 1077.64432132964, 1038.0350877192982,...","[1141.0, 1163.691966759003, 1129.0724391624221...","[279.0, 234.5822714681442, 223.9309564233165, ...","[69.14600550964187, 70.02967359050444, 68.3195..."


There are a few. Let's order them in descending order by `Pay Gap Change (%)`.

In [26]:
df_percent_female_pos_pay_gap_pos_sorted = df_percent_female_pos_pay_gap_pos.sort_values(by=['Pay Gap Change (%)'], ascending=False)
df_percent_female_pos_pay_gap_pos_sorted.iloc[:10]

,Pay Gap Change (%),Total Weekly Pay Change (%),Percent Female Change (%),Pay Gap Change,Total Weekly Pay Change,Percent Female Change,Correlation2,Correlation1,Male Weekly Pay,Female Weekly Pay,Total Weekly Pay,Pay Gap,Percent Female
Occupation,,,,,,,,,,,,,
Postal service mail carriers,652.155378,-12.451824,27.660819,151.733531,-132.414508,8.877628,55.922125,-129.108239,"[983.0, 988.5235457063714, 1067.3344651952461,...","[808.0, 876.86648199446, 974.2043010752689, 90...","[931.0, 967.0116343490306, 1026.5246179966043,...","[175.0, 111.6570637119114, 93.13016411997728, ...","[40.97222222222222, 40.6015037593985, 39.14473..."
Police and sheriffs patrol officers,490.152681,4.435396,25.768434,261.623982,46.377498,2.983258,29.741338,1.479430,"[1131.0, 1048.9617728531857, 1054.777589134125...","[816.0, 1009.0110803324101, 981.5291454442558,...","[1092.0, 1039.7423822714682, 1050.591963780418...","[315.0, 39.95069252077565, 73.24844368986987, ...","[14.560439560439562, 13.662790697674417, 14.10..."
Social and community service managers,161.353330,5.798099,9.445825,172.247964,62.530718,5.967711,128.301908,22.466199,"[1336.0, 1312.2265927977842, 1261.966044142614...","[1057.0, 1077.64432132964, 1038.0350877192982,...","[1141.0, 1163.691966759003, 1129.0724391624221...","[279.0, 234.5822714681442, 223.9309564233165, ...","[69.14600550964187, 70.02967359050444, 68.3195..."
Dispatchers,140.760181,3.831374,11.074380,117.514434,28.892672,5.982143,-7.050660,-14.305597,"[926.0, 771.3556786703602, 768.0622524052066, ...","[725.0, 707.84432132964, 702.1386530843237, 69...","[783.0, 727.3074792243768, 736.6700622524052, ...","[201.0, 63.511357340720224, 65.92359932088289,...","[60.0, 56.86274509803921, 54.83870967741935, 5..."
Counselors,101.486471,3.236633,8.824034,45.835677,29.501850,5.819742,-68.110349,9.330476,"[1004.0, 922.9634349030472, 933.394453876627, ...","[913.0, 915.7927977839337, 949.0905489530277, ...","[941.0, 918.8659279778394, 945.9513299377476, ...","[91.0, 7.170637119113508, -15.696095076400638,...","[71.77304964539007, 72.61538461538461, 71.9939..."
Pharmacists,64.723935,2.451434,32.257733,149.703923,49.554404,15.301745,-57.549987,-118.863048,"[2271.0, 2282.3113573407204, 2193.267685342388...","[1890.0, 1878.7069252077565, 1924.341256366723...","[2071.0, 1969.876454293629, 2013.2857951329938...","[381.0, 403.6044321329639, 268.926428975665, 3...","[62.737642585551335, 57.874015748031496, 59.90..."
Lawyers,56.064417,-8.807359,27.998369,158.065137,-188.040711,8.821404,1.742563,-106.336064,"[2202.0, 2156.3130193905818, 2182.803621958121...","[1762.0, 1795.7324099722994, 1694.131861912846...","[1947.0, 1947.3401662049864, 1985.032823995472...","[440.0, 360.5806094182824, 488.6717600452746, ...","[40.328253223915596, 43.149807938540334, 40.13..."
Driversales workers and truck drivers,49.454250,-1.002738,43.567904,89.342709,-8.275352,1.685315,6.670935,2.602990,"[829.0, 826.6720221606649, 823.5217883418223, ...","[559.0, 603.3578947368421, 659.2359932088285, ...","[817.0, 816.4282548476456, 816.1969439728354, ...","[270.0, 223.3141274238228, 164.28579513299383,...","[5.553565030455034, 4.915134370579914, 4.27198..."
Medical and health services managers,49.130646,4.338401,7.607290,131.119911,58.544782,5.211725,-15.263389,-6.129977,"[1729.0, 1526.3213296398894, 1684.714204867006...","[1331.0, 1205.6914127423825, 1312.193548387096...","[1408.0, 1351.1529085872578, 1467.06168647425,...","[398.0, 320.629916897507, 372.52065647990935, ...","[73.72134038800705, 70.98360655737706, 74.8592..."


TODO: Find percent of `Pay Gap Change`. What do I put on the denominator?

TODO: Analyze
- Police and sheriffs patrol officers
- Lawyers
- Pharmacists
- Counselors

It's good to view problems in a positive light, so here are some improvements that have been made regarding equal pay

Let's look at the general results from all occupations

In [27]:
df.loc['Total fulltime wage and salary workers']

Pay Gap Change (%)                                                      -5.98433
Total Weekly Pay Change (%)                                              4.41442
Percent Female Change (%)                                                1.26232
Pay Gap Change                                                          -11.7121
Total Weekly Pay Change                                                  37.4582
Percent Female Change                                                   0.554706
Correlation2                                                            -0.27009
Correlation1                                                             2.96448
Male Weekly Pay                [973.0, 963.9385041551247, 957.4617996604414, ...
Female Weekly Pay              [789.0, 788.7700831024931, 783.7583474816072, ...
Total Weekly Pay               [886.0, 880.9639889196677, 870.6100735710244, ...
Pay Gap                        [184.0, 175.16842105263163, 173.70345217883425...
Percent Female              

In general the pay gap decreased by 6%, total weekly pay increased by around 4%, and the proportion of women in the work force increased by around 1%.

Are there any occupations in which percent female change is positive, but pay gap change is negative? (The difference between men and women pays decreased.)

In [28]:
df_percent_female_pos_pay_gap_neg = df_percent_female_pos[df_percent_female_pos['Pay Gap Change'] < 0]
print(df_percent_female_pos_pay_gap_neg.shape)
df_percent_female_pos_pay_gap_neg.head()

(33, 13)


,Pay Gap Change (%),Total Weekly Pay Change (%),Percent Female Change (%),Pay Gap Change,Total Weekly Pay Change,Percent Female Change,Correlation2,Correlation1,Male Weekly Pay,Female Weekly Pay,Total Weekly Pay,Pay Gap,Percent Female
Occupation,,,,,,,,,,,,,
Total fulltime wage and salary workers,-5.984335,4.414418,1.262321,-11.712065,37.458179,0.554706,-0.270090,2.964479,"[973.0, 963.9385041551247, 957.4617996604414, ...","[789.0, 788.7700831024931, 783.7583474816072, ...","[886.0, 880.9639889196677, 870.6100735710244, ...","[184.0, 175.16842105263163, 173.70345217883425...","[44.497996833005956, 44.39843915530758, 44.252..."
Management professional and related occupations,-5.329091,2.639106,2.247015,-21.953368,32.037750,1.131191,-2.101778,3.082734,"[1468.0, 1477.1512465373962, 1485.897000565931...","[1078.0, 1077.64432132964, 1074.6593095642331,...","[1246.0, 1253.8371191135736, 1243.130730050933...","[390.0, 399.50692520775624, 411.23769100169784...","[51.47311916079331, 51.672421462918635, 51.543..."
Management business and financial operations occupations,-21.623411,3.023036,4.081821,-101.803849,39.760178,1.815457,-12.860920,3.935588,"[1537.0, 1563.1988919667592, 1560.191850594227...","[1168.0, 1161.643213296399, 1150.0005659309563...","[1355.0, 1359.3479224376733, 1343.585738539898...","[369.0, 401.5556786703603, 410.19128466327106,...","[46.29210089110406, 46.35314721335119, 46.0547..."
Management occupations,-21.051962,2.064575,8.131871,-93.062916,28.905996,3.194061,-24.631875,4.211044,"[1585.0, 1611.3445983379504, 1610.419354838709...","[1236.0, 1201.5939058171746, 1201.274476513865...","[1429.0, 1425.9324099722994, 1433.576683644595...","[349.0, 409.75069252077583, 409.1448783248445,...","[42.472360317578094, 42.524109651454175, 41.42..."
Marketing and sales managers,-2.445243,1.592938,22.960120,-12.282013,24.569948,8.667445,-41.767722,-67.050600,"[1852.0, 1789.5861495844877, 1825.979060554612...","[1362.0, 1319.3972299168977, 1194.996038483305...","[1567.0, 1545.7844875346261, 1536.124504810413...","[490.0, 470.18891966758997, 630.9830220713072,...","[46.41744548286604, 43.05835010060362, 43.7567..."


There are a few. Let's order them in ascending order by `Pay Gap Change (%)`.

In [29]:
df_percent_female_pos_pay_gap_neg_sorted = df_percent_female_pos_pay_gap_neg.sort_values(by=['Pay Gap Change (%)'], ascending=True)
df_percent_female_pos_pay_gap_neg_sorted.iloc[:10]

,Pay Gap Change (%),Total Weekly Pay Change (%),Percent Female Change (%),Pay Gap Change,Total Weekly Pay Change,Percent Female Change,Correlation2,Correlation1,Male Weekly Pay,Female Weekly Pay,Total Weekly Pay,Pay Gap,Percent Female
Occupation,,,,,,,,,,,,,
Office clerks general,-inf,1.448937,2.855027,-31.000000,9.954848,2.391670,1.398700,-1.251263,"[670.0, 799.0138504155126, 739.8092812676854, ...","[701.0, 686.3324099722993, 678.0713073005094, ...","[697.0, 698.6249307479226, 684.3497453310696, ...","[-31.0, 112.68144044321332, 61.737973967176, -...","[86.16216216216216, 84.81283422459893, 84.4444..."
Clinical laboratory technologists and technicians,-193.362779,-13.117970,2.822250,-190.540340,-134.830496,2.034927,-43.028411,26.349535,"[819.0, 1086.8637119113575, 1120.7011884550084...","[911.0, 865.5983379501386, 872.7028862478777, ...","[893.0, 911.6952908587259, 928.1624221844935, ...","[-92.0, 221.26537396121898, 247.99830220713068...","[74.13793103448276, 68.35016835016835, 67.1186..."
Construction and extraction occupations,-83.299081,-1.439482,26.147258,-119.704663,-11.800888,0.617238,-1.560706,-0.086329,"[809.0, 815.4038781163435, 822.4753820033956, ...","[785.0, 821.5501385041553, 668.653650254669, 7...","[808.0, 815.4038781163435, 820.3825693265421, ...","[24.0, -6.146260387811708, 153.82173174872662,...","[2.977860929217337, 2.6028957214901576, 2.5255..."
Bartenders,-71.469263,4.048161,12.902962,-102.704663,22.604737,6.275211,-58.279738,-25.324558,"[601.0, 614.6260387811635, 733.530843237125, 6...","[560.0, 527.5540166204987, 521.1103565365025, ...","[581.0, 567.5047091412744, 629.9366157328806, ...","[41.0, 87.0720221606648, 212.42048670062252, 8...","[54.90909090909091, 53.333333333333336, 48.221..."
Purchasing managers,-67.418460,-10.183571,15.259875,-417.982976,-154.426351,5.904118,-77.976662,-104.907715,"[1431.0, 1454.6149584487537, 1464.968873797396...","[1229.0, 1165.7407202216068, 1223.249009620826...","[1362.0, 1301.9828254847646, 1333.121675155631...","[202.0, 288.87423822714686, 241.71986417657058...","[44.5945945945946, 53.608247422680414, 49.1978..."
Insurance sales agents,-53.056723,-15.526053,16.301977,-270.125093,-157.881569,7.850689,-120.508469,-49.972788,"[1048.0, 1083.7905817174517, 1220.109790605546...","[809.0, 814.3795013850416, 707.3706847764573, ...","[859.0, 918.8659279778394, 877.9349179400114, ...","[239.0, 269.41108033241005, 512.7391058290888,...","[56.00858369098712, 55.00000000000001, 48.4633..."
Nursing psychiatric and home health aides,-50.181917,0.006169,0.134846,-45.328645,0.031828,0.120049,2.820570,-0.261548,"[558.0, 597.2116343490305, 558.7809847198641, ...","[513.0, 505.017728531856, 521.1103565365025, 4...","[516.0, 512.1883656509696, 526.3423882286361, ...","[45.0, 92.19390581717454, 37.6706281833616, 73...","[89.14728682170544, 88.17591925018024, 86.0028..."
Arts design entertainment sports and media occupations,-40.464315,6.510394,1.844083,-104.668394,66.381199,0.797475,8.270572,5.113322,"[1151.0, 1189.3013850415514, 1191.856819468025...","[997.0, 965.9872576177287, 995.132427843803, 9...","[1086.0, 1091.9855955678672, 1088.262591963780...","[154.0, 223.3141274238227, 196.72439162422188,...","[44.04255319148936, 43.02587176602925, 45.0029..."
Sales representatives wholesale and manufacturing,-38.215808,-4.146630,10.584113,-142.263509,-50.225019,2.539095,16.020893,-10.216531,"[1239.0, 1251.7883656509696, 1192.903225806451...","[1009.0, 979.3041551246539, 912.4663271080929,...","[1161.0, 1185.2038781163437, 1095.587436332767...","[230.0, 272.4842105263157, 280.43689869835885,...","[26.52877697841727, 27.873303167420815, 26.940..."


TODO: Analyze
- Purchasing managers/ Insurance sales agents/ Clinical laboratory technologists and technicians
- Legal occupations/ Arts design entertainment sports and media occupations/ Bartenders

TODO: Are there any occupations in which women actually make more than men? Should we do this? I think counselors is an interesting one.  

I can't seem to escape my pessimism...

Are there any occupations in which percent female change is negative? 

In [32]:
df_percent_female_neg = df[df['Percent Female Change'] < 0]
print(df_percent_female_neg.shape)
df_percent_female_neg.head()

(37, 13)


,Pay Gap Change (%),Total Weekly Pay Change (%),Percent Female Change (%),Pay Gap Change,Total Weekly Pay Change,Percent Female Change,Correlation2,Correlation1,Male Weekly Pay,Female Weekly Pay,Total Weekly Pay,Pay Gap,Percent Female
Occupation,,,,,,,,,,,,,
Computer and information systems managers,-10.179370,-2.108592,-14.884284,-21.872687,-40.430792,-4.451795,145.906390,28.055431,"[1920.0, 1943.2426592797785, 1837.489530277306...","[1727.0, 1668.709695290859, 1757.962648556876,...","[1877.0, 1887.9263157894738, 1818.654216185625...","[193.0, 274.5329639889196, 79.52688172043008, ...","[25.45757071547421, 27.77777777777778, 25.3378..."
Property real estate and community association managers,-23.470609,3.021773,-4.242424,-67.777942,29.859363,-2.362869,-27.857649,-38.303709,"[1148.0, 1191.3501385041552, 1210.692133559705...","[927.0, 894.2808864265929, 852.8211658177702, ...","[1018.0, 980.3285318559558, 1003.5036785512167...","[221.0, 297.0692520775623, 357.8709677419356, ...","[53.333333333333336, 57.24465558194775, 54.975..."
Business and financial operations occupations,-24.768544,5.521249,-3.801174,-91.526277,63.625463,-2.148036,6.157721,-27.042388,"[1383.0, 1423.8836565096954, 1436.715902659875...","[1105.0, 1080.7174515235458, 1065.241652518392...","[1216.0, 1202.6182825484766, 1214.877758913412...","[278.0, 343.1662049861495, 371.47425014148257,...","[54.36178543461237, 54.41152922337871, 55.6577..."
Wholesale and retail buyers except farm products,-101.029106,-12.954672,-1.267356,-196.343449,-130.669874,-0.567589,-90.627285,-27.450141,"[878.0, 903.5002770083104, 985.7147707979626, ...","[880.0, 909.646537396122, 800.5008488964346, 1...","[878.0, 907.5977839335181, 876.8885116015846, ...","[-2.0, -6.146260387811594, 185.213921901528, -...","[44.21768707482993, 56.493506493506494, 54.605..."
Management analysts,-48.349527,0.392747,-1.711286,-191.898594,5.993338,-0.764979,11.255559,-68.764871,"[1642.0, 1691.2459833795017, 1659.600452744765...","[1437.0, 1347.05540166205, 1404.2773061686473,...","[1532.0, 1535.5407202216068, 1544.495755517826...","[205.0, 344.1905817174518, 255.32314657611778,...","[43.93700787401575, 42.07920792079208, 43.2478..."


There is a substantial number. Let's sort them in ascending order to `Percent Female Change (%)`.

In [33]:
df_percent_female_neg_sorted = df_percent_female_neg.sort_values(by=['Percent Female Change (%)'], ascending=True)
df_percent_female_neg_sorted.iloc[:10]

,Pay Gap Change (%),Total Weekly Pay Change (%),Percent Female Change (%),Pay Gap Change,Total Weekly Pay Change,Percent Female Change,Correlation2,Correlation1,Male Weekly Pay,Female Weekly Pay,Total Weekly Pay,Pay Gap,Percent Female
Occupation,,,,,,,,,,,,,
Computer support specialists,72.228231,5.985531,-23.582960,41.518135,61.275352,-8.373103,-40.785929,-20.345177,"[1114.0, 1161.643213296399, 1128.0260328239954...","[1015.0, 941.4022160664821, 1061.056027164686,...","[1085.0, 1094.034349030471, 1109.1907187323147...","[99.0, 220.24099722991684, 66.97000565930944, ...","[27.131782945736433, 27.99097065462754, 25.254..."
Advertising sales agents,-126.314167,8.836402,-23.507262,-494.424130,81.027387,-14.365549,372.069231,-164.883012,"[950.0, 916.8171745152356, 1081.98415393322, 1...","[1053.0, 903.5002770083104, 1045.3599320882852...","[998.0, 908.6221606648201, 1070.4736842105262,...","[-103.0, 13.316897506925216, 36.62422184493494...","[46.74556213017752, 54.92227979274611, 39.9014..."
Sales representatives services all other,198.053537,8.803659,-21.547328,273.769800,99.523316,-7.268357,-113.676572,-59.217526,"[1345.0, 1364.469806094183, 1257.7804187889078...","[933.0, 923.9878116343491, 864.331635540464, 7...","[1230.0, 1174.9601108033241, 1088.262591963780...","[412.0, 440.4819944598339, 393.44878324844376,...","[26.46370023419204, 26.200873362445414, 33.957..."
Computer programmers,11.657996,9.153838,-20.146964,22.656551,133.424130,-5.364334,30.396561,-29.639142,"[1662.0, 1576.5157894736844, 1534.031692133559...","[1445.0, 1396.2254847645431, 1372.885116015846...","[1591.0, 1519.1506925207757, 1462.876061120543...","[217.0, 180.2903047091413, 161.14657611771372,...","[21.261682242990652, 21.04018912529551, 25.806..."
Electrical electronics and electromechanical assemblers,29.589681,0.466468,-20.024737,42.926721,2.809030,-12.064083,93.335595,-5.617482,"[706.0, 698.6249307479226, 646.679117147708, 5...","[518.0, 601.3091412742383, 579.7091114883984, ...","[605.0, 639.21108033241, 617.37973967176, 587....","[188.0, 97.31578947368428, 66.97000565930955, ...","[48.18181818181818, 40.65040650406504, 47.6190..."
Installation maintenance and repair occupations,79.489501,1.402871,-18.394920,50.043671,12.921540,-0.747519,-1.472666,3.597277,"[936.0, 904.5246537396123, 903.0486700622524, ...","[823.0, 753.9412742382273, 819.3361629881155, ...","[934.0, 899.4027700831026, 900.955857385399, 8...","[113.0, 150.58337950138502, 83.71250707413697,...","[3.3162073797290983, 3.3863636363636367, 3.427..."
Computer and information systems managers,-10.179370,-2.108592,-14.884284,-21.872687,-40.430792,-4.451795,145.906390,28.055431,"[1920.0, 1943.2426592797785, 1837.489530277306...","[1727.0, 1668.709695290859, 1757.962648556876,...","[1877.0, 1887.9263157894738, 1818.654216185625...","[193.0, 274.5329639889196, 79.52688172043008, ...","[25.45757071547421, 27.77777777777778, 25.3378..."
Inspectors testers sorters samplers and weighers,-33.925489,-1.341000,-12.047251,-97.040711,-10.479645,-4.837578,-2.123211,-31.886253,"[868.0, 873.7933518005541, 872.7028862478777, ...","[679.0, 652.5279778393352, 611.1013016411998, ...","[771.0, 783.6481994459834, 747.1341256366724, ...","[189.0, 221.26537396121887, 261.6015846066779,...","[35.317460317460316, 38.35425383542538, 35.131..."
Other teachers and instructors,-8.666847,2.679386,-10.605457,-24.197631,24.972613,-6.573930,-35.399780,-44.968280,"[1095.0, 1155.4969529085874, 1187.671194114318...","[840.0, 839.9889196675902, 822.4753820033956, ...","[957.0, 918.8659279778394, 981.5291454442558, ...","[255.0, 315.5080332409973, 365.1958121109225, ...","[55.412371134020624, 63.24582338902148, 57.837..."


There seems to be more engineering occupations in this category!! A few ones to delve into:
- Computer programmers
- Sales representatives services all other
- Advertising sales agents